# 03. DQN example with CartPole

## Colab 용 package 설치 코드

In [ ]:
!pip install gym
!pip install JSAnimation

### package import

In [ ]:
# The typical imports
from IPython.display import clear_output
import gym
import numpy as np
import matplotlib.pyplot as plt
import random

from collections import deque

%matplotlib inline

import tensorflow as tf

np.random.seed(285)
tf.set_random_seed(285)

print("tensorflow version: ", tf.__version__)
print("gym version: ", gym.__version__)

### 게임 화면을 보여주기 위한 함수

In [ ]:
# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    #plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))

### 그래프를 그리기 위한 함수

In [ ]:
def plot(frame_idx, episode, rewards, losses):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('episode %s. reward: %s' % (episode, np.mean(rewards[-10:])))
    plt.plot(rewards)
    plt.subplot(132)
    plt.title('loss')
    plt.plot(losses)
    plt.show()

<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.4/MathJax.js?config=TeX-MML-AM_CHTML' async></script>
## CartPole
CartPole is game that ballance pole on the car.  
this game's observation is $x$, $x\prime$, $\theta$, $\theta\prime$

$x$ : 카트의 위치  
$\theta$ : 막대의 각도  
$x\prime$ : 카트의 속도  
$\theta\prime$ : 막대의 각속도  

Action is **Left** or **Right**  

모든 step 마다 보상을 1 받으며, 아래 3가지 경우에 episode가 끝난다.
1. 카트가 바깥으로 나갈 때
2. 기둥이 너무 많이 기울었을 때
3. 200 step 지났을 때  
<img src="./img/cartpole.gif" width="60%" align="left"> 

In [ ]:
# CartPole 환경
env_id = "CartPole-v0"
env = gym.make(env_id)

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
print("Observation size : ", state_size)
print("Action size : ", action_size)

## DQN Agent

### Replay Buffer

In [ ]:
from collections import deque

state = env.reset()
action = env.action_space.sample()
next_state, reward, done, _ = env.step(action)

print("state_size:", np.shape(state))
print("next_state_size:", np.shape(next_state))

# state, next_state의 차원을 늘림. batch 단위로 묶을 때 size를 맞추기 위해
# 참고) np.expand_dims( array, axis=0 )
state = 
next_state = 

print("state_size:", np.shape(state))
print("next_state_size:", np.shape(next_state))

In [ ]:
#  deque buffer 생성
buffer = deque(maxlen=1000)

# buffer에 transition((state, action, reward, next_state, done))을 append
transition = 


print(np.shape(buffer))
print(buffer)

In [ ]:
# buffer에서 batch size만큼 sampling
for i in range(5):
    buffer.append(transition)
    
# 참고) random.sample(list, sample 수)
transitions = 
print(transitions)

state, action, reward, next_state, done = [], [], [], [], []
for t in transitions:
    # 각 리스트에 transition의 요소들을 하나씩 append
    # state
    # action
    # reward
    # next_state
    # done
    
state = np.squeeze(state)
next_state = np.squeeze(next_state)

print(np.shape(state))
print(next_state)
print(action)
print(reward)
print(done)

### Replay Buffer class

In [ ]:
# Unifrom Replay Buffer
class Replay_Buffer(object):
    def __init__(self, capacity):
        # 직접 작성해보세요
        self.buffer = 
    
    # buffer 길이 체크
    def __len__(self):
        return len(self.buffer)
    
    # buffer에 sample 추가
    def add_sample(self, state, action, reward, next_state, done):
        # 직접 작성해보세요.
        state      = 
        next_state = 
            
        # buffer에 append
    
    # buffer에서 batch_size만큼 뽑기
    def sampling(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return np.concatenate(state), action, reward, np.concatenate(next_state), done

### DQN Agent Class
<img src="./img/hyperparameters.png" width="100%" align="left"> 

Q Learning에서 Q함수의 업데이트식은 다음과 같다.
$$Q(s,a) \gets Q(s,a) +  \alpha [r + \gamma Q(s \prime, argmax_{a\prime} Q(s \prime,  a\prime) - Q(s,a)]$$  
DQN에서는 업데이트식에서 TD error 부분을 Loss로 보고 학습한다.
$$ Loss = E [(y - Q(s,a))^{2}]$$

In [ ]:
layer = tf.contrib.layers

class DQNAgent:
    def __init__(self, sess, state_size, action_size):
        self.sess = sess    
        
        self.state_size = state_size
        self.action_size = action_size
        
        # hyper parameter
        self.batch_size = 32
        self.s_epsilon = 1
        self.e_epsilon = 0.01
        self.n_epsilon_decay = 1000
        self.epsilon = self.s_epsilon
        self.discount_factor = 0.99
        self.learning_rate = 0.00025
        
        # set placeholder
        self.input_p_net = tf.placeholder(tf.float32, shape=(None, self.state_size))
        self.input_t_net = tf.placeholder(tf.float32, shape=(None, self.state_size))
        
        # action placeholder used to get main Q(S,A)
        self.input_A = tf.placeholder(tf.int32, shape=None)
        
        # target Y placeholder
        self.input_Y = tf.placeholder(tf.float32, shape=None)
        
        self.policy_q = self._build_network(self.input_p_net, "policy_net")
        self.target_q = self._build_network(self.input_t_net, "target_net")
        
        self.loss_op, self.train_op = self._build_op()
        
    def _build_network(self, inputs, name):
        """
        tf.contrib.layers.fully_connected()를 이용해
        hidden layer가 하나인 신경망을 구성합니다.
        입력 : 상태 (state_size)
        출력 : q-value (action_size)
        hidden layer size : 128
        activation function : Relu
        """
        # 빈칸 {} 을 지우고 채워주세요.
        # 참고) layer.fully_connected(입력, 출력 사이즈, activation function)
        # 참고2) relu -> tf.nn.relu
        with tf.variable_scope(name):          
            fc1 = layer.fully_connected({}, {}, activation_fn={})
            fc2 = layer.fully_connected({}, {}, activation_fn={})
            q_value = layer.fully_connected({}, {}, activation_fn=None)
        return q_value
    
    def _get_Q_with_A(self, Q_value):
        """
        Q(S,A) 를 구합니다. [batch_size,]
        Q 신경망을 이용해 특정 action에 해당하는 Q-value를 구합니다.
        """
        # 직접 작성해보세요.
        # 참고) # 이전 실습에서 현재 action에 대한 Q_value 구하는 연산
        #          curr_action = tf.one_hot(input_action, action_size)
        #          curr_q_value = tf.reduce_sum(tf.multiply(q_value, curr_action))
        one_hot = 
        Q_with_A = 
        return Q_with_A

    def _get_Y(self, next_state, reward, done):
        """
        학습 시 Target이 되는 Y값을 구합니다. (Reward + 다음 state의 Q값들 중 max)
        """

        # 참고)
        # # episode가 끝났다면
        # if done:
        #         next_q_value = np.array([reward])
        # # 끝나지 않았다면
        # else:
        #         next_q_value = reward + gamma * np.max(sess.run(q_value, feed_dict={inputs: next_state}), axis=1)

        Y = 
        return Y
    
    def _build_op(self):
        """
        신경망 학습을 위한 Loss function과 Optimaizer를 정의합니다.
        """
        # 참고) 이전 실습에서 Loss 함수 구성
        #         loss_op = tf.square(target - curr_q_value)
        #         opt = tf.train.GradientDescentOptimizer(learning_rate=0.1)
        #         train_op = opt.minimize(loss_op)
        main_Q_with_A = 
        loss_op = 
        train_op = tf.train.RMSPropOptimizer(learning_rate=self.learning_rate,
                                                              decay=0.95,
                                                              momentum=0.95,
                                                              epsilon=0.01).minimize(loss_op)
        return loss_op, train_op
    
    def train(self):
        """
        replay buffer에서 batch size만큼 가져온 후
        학습 네트워크를 학습합니다.
        loss function은 위의 수식 참고
        """
        # replay buffer로부터 transition을 가져옴
        # 직접 작성해보세요.
        # 참고) 위에 작성한 replay buffer class, replay_buffer.sampling()
        state, action, reward, next_state, is_terminal = 
        
        action = np.squeeze(action, 1)

        # get target Y: [batch_size,]
        # 내장 함수 get_Y를 이용함.
        Y = 
        loss, _ = self.sess.run([{}, {}], feed_dict={{}})
        return loss
    
    def update_target_network(self):
        """
        학습 네트웍의 변수의 값들을 타겟 네트웍으로 복사해서 타겟 네트웍의 값들을 최신으로 업데이트합니다.
        """
        copy_op = []

        main_q_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='policy_net')
        target_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='target_net')

        for main_q_var, target_var in zip(main_q_vars, target_vars):
            copy_op.append(target_var.assign(main_q_var.value()))
        self.sess.run(copy_op)
        
    def get_action(self, state, is_play=False):
        """
        epsilon-greedy로 action을 선택합니다.
        """
        if is_play:
            self.epsion = 0
        
        # epsilon greedy policy
        # 직접 작성해보세요.
        if self.epsilon > random.random():
            action = 
        else:
            # 참고) policy_q 에 state를 입력해 나온 출력중 max 인것 선택
            # 참고2) 입력 사이즈를 맞추기 위해 state에 추가 처리를 해야함
            action = 
        
        # 매 step마다 epsilon을 줄여나갑니다.
        if self.epsilon >= self.e_epsilon:
            self.epsilon -= (self.s_epsilon - self.e_epsilon)/self.n_epsilon_decay    
        return action

### DQN agent train

In [ ]:
# Session 열기
sess = tf.Session()

# DQN Agent 객체 생성
agent = DQNAgent(sess, state_size, action_size)

# 변수 초기화
sess.run(tf.global_variables_initializer())

### DQN 학습

In [ ]:
EPISODE = 30

replay_initial = 50
target_update = 10

total_step = 1
all_episode_reward = []
losses = []

replay_buffer = Replay_Buffer(capacity=1000)
for e in range(EPISODE):
    print("EPISODE: {}".format(e+1))
    observation = env.reset()
    done = False
   
    step = 1
    episode_reward = 0
    frames = []
    
    while not done:
        # 직접 작성해보세요.
        # action 선택
        action = 
        
        # 선택한 action으로 env.step()
        next_observation, reward, done, _ = env.step(action)
        step += 1
        total_step += 1
        episode_reward += reward
        
        # trajectory(S, A, R, S', done)를 Replay buffer에 저장 
        # replay buffer에 저장하는 코드 작성
        
        observation = next_observation
        
        # 만약에 episode가 끝났으면 reward 저장
        if done:
            all_episode_reward.append(episode_reward)
        
        # replay buffer가 일정 이상 채워지면 학습 시작
        if len(replay_buffer) > replay_initial:
            # 신경망 업데이트 코드 작성
            loss = 
            losses.append(loss)
            
        # 일정 step마다 target Q 업데이트
        if total_step > replay_initial and total_step % target_update == 0:
            # policy network를 target network에 복사해주는 코드 작성
            
        # 그래프 그리기
        if total_step % 100 == 0:
            plot(step, e, all_episode_reward, losses)
            
    print(total_step)
    env.close()

### 학습된 DQN 테스트